# Lab 1 — Data Cleaning + Linear Regression from Scratch (Normal Equation & Gradient Descent)

**Course:** Applied Machine Learning  
**Duration:** ~2 hours (in-lab)

## Learning goals
By the end of this lab, you should be able to:
- Diagnose realistic data issues using summaries and plots.
- Fix issues column-by-column and verify that your fixes work.
- Build **univariate** and **multivariate** linear regression models.
- Solve univariate linear regression using:
  - the **Normal Equation** (closed form)
  - **Gradient Descent** (iterative optimization)
- Compare model performance on:
  - corrupted raw data
  - corrupted **cleaned** data
  - clean “oracle” data

> **Important:** In the first half of the lab, you must work **only** with the corrupted dataset.

## 0) Setup (runs in Colab / local)

This notebook is **self-contained**:
- If `Power_plant.csv` is available locally, we use it.
- Otherwise, we download the Combined Cycle Power Plant dataset from UCI (zip) and load it.

We then create two CSV files:
- `power_plant_clean.csv` (oracle)
- `power_plant_corrupt.csv` (contains realistic corruption you must fix)

In [14]:
# If you run this in Google Colab, you may want to install dependencies (usually already installed):
# !pip -q install pandas numpy matplotlib seaborn scikit-learn

import os, zipfile, urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# If you run this in Google Colab, you may want to install dependencies (usually already installed):
# !pip -q install pandas numpy matplotlib scikit-learn openpyxl

import os, zipfile, urllib.request
import numpy as np
import pandas as pd

RANDOM_STATE = 42
rng = np.random.default_rng(RANDOM_STATE)

# Where the rest of the lab expects the "clean/original" dataset to be:
DATA_LOCAL = "Power_plant.csv"          # stable local name used throughout the lab
CLEAN_OUT  = "power_plant_clean.csv"    # optional alias if you also want this file

UCI_ZIP_URL  = "https://archive.ics.uci.edu/static/public/294/combined+cycle+power+plant.zip"
UCI_ZIP_PATH = "ccpp.zip"

EXPECTED = ["AT", "V", "AP", "RH", "PE"]

def ensure_openpyxl():
    # Defensive: if openpyxl is missing, install it (Colab usually has it)
    try:
        import openpyxl  # noqa: F401
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", "openpyxl"])

def extract_best_table_file_from_zip(zip_path: str) -> str:
    """
    Extract the best candidate dataset file from the zip and return the extracted path.
    Prefers:
      1) Folds5x2_pp.xlsx (or similarly named)
      2) any .xlsx
      3) Folds5x2_pp.csv
      4) any .csv
    """
    with zipfile.ZipFile(zip_path, "r") as z:
        names = z.namelist()

        def pick(preferred_substr: str, exts: tuple[str, ...]):
            cand = [n for n in names if n.lower().endswith(exts)]
            pref = [n for n in cand if preferred_substr in n.lower()]
            return (pref[0] if pref else (cand[0] if cand else None))

        chosen = (
            pick("folds5x2_pp", (".xlsx",))
            or pick("", (".xlsx",))
            or pick("folds5x2_pp", (".csv",))
            or pick("", (".csv",))
        )

        if chosen is None:
            raise RuntimeError(f"No .xlsx or .csv found in zip. Contents:\n{names}")

        print("Extracting:", chosen)
        z.extract(chosen, ".")
        return chosen  # note: may include subfolder, e.g. "CCPP/Folds5x2_pp.xlsx"

def load_table(path: str) -> pd.DataFrame:
    """
    Load a table from .xlsx or .csv, normalize columns to EXPECTED when possible.
    """
    if path.lower().endswith(".xlsx"):
        ensure_openpyxl()
        df = pd.read_excel(path)
    else:
        # robust CSV read
        df = pd.read_csv(path, encoding="utf-8-sig")

    # Some versions have whitespace in headers
    df.columns = [str(c).strip() for c in df.columns]

    # If expected columns exist, keep them in correct order
    if all(c in df.columns for c in EXPECTED):
        return df[EXPECTED].copy()

    # Otherwise: sometimes Excel has unnamed columns or different header row.
    # Try a simple recovery: if there are 5 numeric columns, assume order is AT,V,AP,RH,PE.
    if df.shape[1] >= 5:
        df5 = df.iloc[:, :5].copy()
        df5.columns = EXPECTED
        return df5

    raise RuntimeError(f"Could not match expected columns {EXPECTED}. Got columns: {df.columns.tolist()}")

def ensure_clean_dataset() -> pd.DataFrame:
    # 1) Use local file if present
    if os.path.exists(DATA_LOCAL):
        print("Using local file:", DATA_LOCAL)
        df = pd.read_csv(DATA_LOCAL, encoding="utf-8-sig")
        return df

    # 2) Otherwise download zip from UCI (if needed)
    if not os.path.exists(UCI_ZIP_PATH):
        print("Downloading dataset from UCI...")
        urllib.request.urlretrieve(UCI_ZIP_URL, UCI_ZIP_PATH)

    # 3) Extract best file (xlsx/csv)
    extracted_rel = extract_best_table_file_from_zip(UCI_ZIP_PATH)

    # 4) Load it
    if not os.path.exists(extracted_rel):
        # extremely defensive; extract should have created it
        raise RuntimeError(f"Extracted file not found on disk: {extracted_rel}")

    df = load_table(extracted_rel)

    # 5) Save to stable filenames that the rest of the lab expects
    df.to_csv(DATA_LOCAL, index=False)
    df.to_csv(CLEAN_OUT, index=False)

    print("Saved:", DATA_LOCAL, "and", CLEAN_OUT)
    print("Dataset shape:", df.shape)
    return df

df_clean = ensure_clean_dataset()

# Final sanity check
assert all(c in df_clean.columns for c in EXPECTED), f"Expected columns {EXPECTED}, got {df_clean.columns.tolist()}"

df_clean.head()


Using local file: Power_plant.csv


,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


### Cleaning Tool B — Fix obvious physical constraints

Engineering priors:
- Relative humidity `RH` should be in **[0, 100]**
- Exhaust vacuum `V` should be **non-negative**

**TODO:** implement one or more fixes:
- Fix unit mistakes (if RH appears in 0..1)
- Replace impossible values with NaN (so they can be imputed)

### Cleaning Tool B — Fix obvious physical constraints

Engineering priors:
- Relative humidity `RH` should be in **[0, 100]**
- Exhaust vacuum `V` should be **non-negative**

**TODO:** implement one or more fixes:
- Fix unit mistakes (if RH appears in 0..1)
- Replace impossible values with NaN (so they can be imputed)

## 1) Create the *corrupted* dataset 

We create a corrupted copy to simulate real-world issues:
- **Non-numeric values** in numeric columns (`"1010,84"`, `"N/A"`, whitespace)
- **Missing values**
- **Impossible ranges** (e.g., RH > 100, negative vacuum)
- **Outliers** (rare spikes)

The goal is to detect and fix these issues **without being told which columns are affected**.

In [ ]:
def make_corrupt_copy(df, seed=42):
    rng = np.random.default_rng(seed)
    d = df.copy(deep=True)
    n = len(d)

    # --- 1) AT: missing + outliers + "N/A" strings (AT is informative for PE)
    m_nan = rng.random(n) < 0.05
    d.loc[m_nan, "AT"] = np.nan

    m_na = rng.random(n) < 0.02
    d.loc[m_na, "AT"] = np.nan  # we'll later also inject a string in a separate step

    m_out = rng.random(n) < 0.02
    d.loc[m_out, "AT"] = d.loc[m_out, "AT"] + rng.choice([25.0, -20.0, 40.0], size=m_out.sum())

    # Inject some "N/A" and whitespace strings in AT (forces object dtype later)
    m_str = rng.random(n) < 0.01
    d.loc[m_str, "AT"] = d.loc[m_str, "AT"].map(lambda x: " N/A " if rng.random() < 0.5 else f" {x:.2f} ")

    # --- 2) AP: decimal comma strings (e.g., "1010,84")
    m_comma = rng.random(n) < 0.03
    d.loc[m_comma, "AP"] = d.loc[m_comma, "AP"].map(lambda x: f"{x:.2f}".replace(".", ","))

    # --- 3) RH: unit mistake and impossible values
    m_unit = rng.random(n) < 0.03
    d.loc[m_unit, "RH"] = d.loc[m_unit, "RH"] / 100.0  # 0..1 instead of 0..100

    m_hi = rng.random(n) < 0.01
    d.loc[m_hi, "RH"] = d.loc[m_hi, "RH"] + 60.0       # >100

    # --- 4) V: negative sign mistakes + mild noise
    m_neg = rng.random(n) < 0.02
    d.loc[m_neg, "V"] = -d.loc[m_neg, "V"]

    d["V"] = d["V"] + rng.normal(0, 0.2, size=n)

    return d

df_corrupt = make_corrupt_copy(df_clean, seed=RANDOM_STATE)
df_corrupt.to_csv("power_plant_corrupt.csv", index=False)

print("Saved two files:")
print(" - power_plant_clean.csv")
print(" - power_plant_corrupt.csv")
df_corrupt.head()

## 2) Work ONLY with the corrupted dataset (first half)

Load the corrupted dataset and do quick diagnostics.

### Task 2.1 — What looks wrong?
Use:
- `df.info()` and `df.describe(include="all")`
- Missingness table
- Range checks (e.g., humidity should be between 0 and 100)
- Plots (pairplot, histograms, boxplots)

Write down at least **3 concrete issues** you suspect exist.

In [ ]:
df = pd.read_csv("power_plant_corrupt.csv")

print("Shape:", df.shape)
display(df.head())

print("\n--- info() ---")
display(df.info())

print("\n--- describe(include='all') ---")
display(df.describe(include="all"))

# Missingness
missing = df.isna().mean().sort_values(ascending=False)
display(pd.DataFrame({"missing_rate": missing}))

# Quick range checks (some will fail if columns are non-numeric)
def safe_to_numeric(s):
    return pd.to_numeric(s.astype(str).str.strip().str.replace(",", ".", regex=False), errors="coerce")

def coerce_numeric_series(s):
    # Make strings, strip, replace decimal comma, set known NA tokens to NaN
    s2 = s.astype(str).str.strip()
    s2 = s2.replace({"N/A": np.nan, "na": np.nan, "NA": np.nan, "": np.nan, "nan": np.nan})
    s2 = s2.str.replace(",", ".", regex=False)
    return pd.to_numeric(s2, errors="coerce")


tmp = df.copy()
for c in ["AT","V","AP","RH","PE"]:
    tmp[c] = safe_to_numeric(tmp[c])

checks = {
    "V_negative_count": int((tmp["V"] < 0).sum()),
    "RH_out_of_range_count": int(((tmp["RH"] < 0) | (tmp["RH"] > 100)).sum()),
    "AT_extreme_count_(< -10 or > 60)": int(((tmp["AT"] < -10) | (tmp["AT"] > 60)).sum()),
    "non_numeric_entries_any_feature": int(tmp[["AT","V","AP","RH"]].isna().sum().sum() - df[["AT","V","AP","RH"]].isna().sum().sum()),
}
display(pd.DataFrame([checks]))

### Task 2.2 — Visual diagnostics

The plots below should help you spot:
- strange distributions (e.g., impossible values)
- outliers
- broken linear relationships

In [ ]:
# Pairplot (sample to keep it fast)
# Scatter-matrix (pairplot alternative without seaborn)
from pandas.plotting import scatter_matrix

sample = df.sample(800, random_state=RANDOM_STATE)
for c in ["AT","V","AP","RH","PE"]:
    sample[c] = safe_to_numeric(sample[c])

axes = scatter_matrix(sample[["AT","V","AP","RH","PE"]], figsize=(10,10), diagonal="hist", alpha=0.6)
plt.suptitle("Scatter matrix (sampled)", y=1.02)
plt.show()

# Histograms + boxplots (numeric coercion)
fig, axes = plt.subplots(2, 4, figsize=(16, 6))
cols = ["AT","V","AP","RH"]
for i, c in enumerate(cols):
    ax = axes[0, i]
    ax.hist(tmp[c].dropna().values, bins=40)
    ax.set_title(f"{c} histogram")
for i, c in enumerate(cols):
    ax = axes[1, i]
    ax.boxplot(tmp[c].dropna().values, vert=True)
    ax.set_title(f"{c} boxplot")
plt.tight_layout()
plt.show()

# Scatter AT vs PE (often strongest single feature)
plt.figure(figsize=(6,4))
plt.scatter(tmp["AT"], tmp["PE"], s=8)
plt.xlabel("AT")
plt.ylabel("PE")
plt.title("AT vs PE (corrupted, coerced to numeric)")
plt.show()

## 3) Cleaning, step-by-step 

You will apply **three generic cleaning tools**.  
Your job is to decide **which column(s)** they should apply to, and **verify** using prints/plots that your fix worked.

You should see:
- fewer NaNs after coercion/imputation
- fewer impossible values (e.g., RH outside 0–100, negative V)
- improved regression performance later

### Cleaning Tool A — Convert to numeric robustly

Problem pattern:
- numeric values stored as strings
- decimal comma `"1010,84"`
- whitespace `" 1010.84 "`
- `"N/A"` / `"na"` entries

**TODO:** choose which columns to coerce.

In [ ]:
df_work = pd.read_csv("power_plant_corrupt.csv")

def coerce_numeric_series(s: pd.Series) -> pd.Series:
    """
    Convert a column to numeric in a robust way:
    - strip whitespace
    - replace decimal comma with dot
    - map common NA tokens to NaN
    - coerce invalid values to NaN
    """
    s2 = s.astype(str).str.strip()
    s2 = s2.replace({"N/A": np.nan, "na": np.nan, "NA": np.nan, "": np.nan, "nan": np.nan})
    s2 = s2.str.replace(",", ".", regex=False)
    return pd.to_numeric(s2, errors="coerce")

# TODO:
# Based on the inspection, select columns that should be numeric
# but currently contain non-numeric entries.
cols_to_coerce = ["AT", "V", "AP", "RH", "PE"]  # edit if needed

for c in cols_to_coerce:
    df_work[c] = coerce_numeric_series(df_work[c])

print("Dtypes after coercion:")
display(df_work.dtypes)

print("NaN counts after coercion (features):")
display(df_work[["AT","V","AP","RH"]].isna().sum().to_frame("nan_count"))


### Cleaning Tool B — Fix obvious physical constraints

Engineering priors:
- Relative humidity `RH` should be in **[0, 100]**
- Exhaust vacuum `V` should be **non-negative**

**TODO:** implement one or more fixes:
- Fix unit mistakes (if RH appears in 0..1)
- Replace impossible values with NaN (so they can be imputed)

In [ ]:
# Numeric copy for checks (do NOT modify num directly)
num = df_work.copy()
for c in ["AT","V","AP","RH","PE"]:
    num[c] = coerce_numeric_series(num[c])

print("Before constraints:")
display(pd.DataFrame([{
    "V_negative": int((num["V"] < 0).sum()),
    "RH_out_of_range": int(((num["RH"] < 0) | (num["RH"] > 100)).sum()),
    "RH_leq_1_fraction": float((num["RH"] <= 1.0).mean())
}]))

# --- Apply constraint fixes to df_work ---

# TODO 1:
# If RH seems to be in [0, 1], convert it to percentage [0, 100].
# (We use a simple heuristic threshold.)
if float((num["RH"] <= 1.0).mean()) > 0.5:
    df_work["RH"] = df_work["RH"] * 100

# TODO 2:
# Replace physically invalid values with NaN:
# - negative V
# - RH outside [0, 100]
df_work.loc[df_work["V"] < 0, "V"] = np.nan
df_work.loc[(df_work["RH"] < 0) | (df_work["RH"] > 100), "RH"] = np.nan

# Recompute checks
num2 = df_work.copy()
for c in ["AT","V","AP","RH","PE"]:
    num2[c] = coerce_numeric_series(num2[c])

print("After constraints:")
display(pd.DataFrame([{
    "V_negative": int((num2["V"] < 0).sum()),
    "RH_out_of_range": int(((num2["RH"] < 0) | (num2["RH"] > 100)).sum()),
    "RH_leq_1_fraction": float((num2["RH"] <= 1.0).mean())
}]))


### Cleaning Tool C — Missing values + outliers

Strategy:
1) Convert all feature columns to numeric (coerce)
2) Replace impossible values with NaN (from tool B)
3) **Impute** missing values (median is a robust baseline)
4) **Clip** extreme outliers using quantiles (winsorization)

**TODO:** choose quantiles (e.g., 1%–99% or 0.5%–99.5%)

In [ ]:
# Freeze a stable base for this step (after coercion + constraints)
df_base_cleaning = df_work.copy()

# Start fresh from the same base every time you run this cell
df_work = df_base_cleaning.copy()

print("NaNs before imputation:")
display(df_work[["AT","V","AP","RH"]].isna().sum().to_frame("nan_count"))

# Median imputation (features only)
for c in ["AT","V","AP","RH"]:
    df_work[c] = df_work[c].fillna(df_work[c].median())

# TODO:
# Choose ONE clipping level below.
# A) mild:    0.005, 0.995
# B) default: 0.01,  0.99
# C) strong:  0.05,  0.95
q_low, q_high = 0.01, 0.99  # edit this line

for c in ["AT","V","AP","RH"]:
    lo, hi = df_work[c].quantile([q_low, q_high])
    df_work[c] = df_work[c].clip(lo, hi)

print("NaNs after imputation:")
display(df_work[["AT","V","AP","RH"]].isna().sum().to_frame("nan_count"))

display(
    df_work[["AT","V","AP","RH"]]
    .describe()
    .loc[["min","max","mean","std"]]
)


## 4) Regression tasks (univariate + multivariate)

You will now:
1) Fit **univariate** linear regression using `AT → PE`  
2) Fit **multivariate** linear regression using `[AT, V, AP, RH] → PE`


To evaluate regression performance, we use the **Root Mean Squared Error (RMSE)**:

$$
\mathrm{RMSE}
=
\sqrt{
\frac{1}{m}
\sum_{i=1}^{m}
\left(
\hat{y}^{(i)} - y^{(i)}
\right)^2
}
$$

where:
- $m$ is the number of samples,
- $y^{(i)}$ is the true target value,
- $\hat{y}^{(i)}$ is the model prediction.

RMSE measures the typical prediction error in the **same physical units as the target variable**.
Lower RMSE indicates better predictive performance.


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Work dataset is your cleaned df_work (from above)
df_model = df_work.copy()

X_uni = df_model[["AT"]].to_numpy()
X_multi = df_model[["AT","V","AP","RH"]].to_numpy()
y = df_model[["PE"]].to_numpy()

# Split ONCE (same rows for both models)
idx = np.arange(len(df_model))
idx_train, idx_test = train_test_split(idx, test_size=0.2, random_state=RANDOM_STATE)

Xuni_train, Xuni_test = X_uni[idx_train], X_uni[idx_test]
Xmul_train, Xmul_test = X_multi[idx_train], X_multi[idx_test]
y_train, y_test       = y[idx_train], y[idx_test]

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1, 1)
    y_pred = np.asarray(y_pred).reshape(-1, 1)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

print("Train size:", len(y_train), "Test size:", len(y_test))

# Standardize univariate AT using TRAIN statistics (GD stability)
x_mu = float(Xuni_train.mean())
x_sigma = float(Xuni_train.std()) if float(Xuni_train.std()) > 0 else 1.0

Xuni_train_s = (Xuni_train - x_mu) / x_sigma
Xuni_test_s  = (Xuni_test  - x_mu) / x_sigma

# Design matrices for GD (bias + scaled x)
m_train = Xuni_train_s.shape[0]
m_test  = Xuni_test_s.shape[0]
Xb_train_gd = np.hstack([np.ones((m_train,1)), Xuni_train_s])
Xb_test_gd  = np.hstack([np.ones((m_test,1)),  Xuni_test_s])


### 4.1 Univariate regression (Normal Equation) — **complete the missing steps**

Model:
$$
\hat{y} = \theta_0 + \theta_1 x
$$

Design matrix with bias term:
$$
X_b =
\begin{bmatrix}
1 & x^{(1)} \\
\vdots & \vdots \\
1 & x^{(m)}
\end{bmatrix}
$$

Normal equation:
$$
\theta = (X_b^T X_b)^{-1} X_b^T y
$$

---

### Matrix operations in Python (NumPy)

In NumPy, matrix operations closely follow the mathematical notation:

- **Matrix multiplication** uses the `@` operator  
  $$
  AB \;\;\leftrightarrow\;\; A \ @ \ B
  $$

- **Transpose** is accessed via `.T`  
  $$
  X_b^T \;\;\leftrightarrow\;\; X_b.T
  $$

- **Matrix inverse** is computed using NumPy’s linear algebra module  
  $$
  (X_b^T X_b)^{-1} \;\;\leftrightarrow\;\; \texttt{np.linalg.inv(Xb.T @ Xb)}
  $$

Putting this together, the normal equation can be written in Python as:
```python
theta = np.linalg.inv(Xb.T @ Xb) @ (Xb.T @ y)
```

---


For problems with a large number of features, computing the normal equation is computationally expensive  
(due to the matrix inverse) and can become numerically unstable.

In practice, one instead uses **iterative optimization methods** (e.g. gradient descent) or  
high-level implementations such as `LinearRegression` in `scikit-learn`.


In [ ]:
m_train, m_test = Xuni_train.shape[0], Xuni_test.shape[0]

# TODO(1): Build the design matrices X_b
# - Add a bias (intercept) column of ones
# - The bias column should be the FIRST column
#
# Hint:
#   Use np.ones(...) and np.hstack(...)
Xb_train = None
Xb_test  = None


# Ensure column-vector shape for y
y_train_col = y_train.reshape(-1, 1)
y_test_col  = y_test.reshape(-1, 1)


# TODO(2): Compute the parameter vector theta using the normal equation
#
#   theta = (X_b^T X_b)^{-1} X_b^T y
#
# Hint:
#   - Use @ for matrix multiplication
#   - Use .T for transpose
theta_ne = None


# --- Evaluation ---
y_pred_uni = Xb_test @ theta_ne
print("Normal Equation RMSE:", rmse(y_test_col, y_pred_uni), "\nR^2:", r2_score(y_test_col, y_pred_uni))


# --- Visualization ---
x_plot = Xb_train[:, 1]
idx = np.argsort(x_plot)

plt.figure(figsize=(6, 4))
plt.scatter(x_plot, y_train_col, label="Training data")
plt.plot(x_plot[idx], (Xb_train @ theta_ne)[idx], linewidth=2, label="Normal Eq fit")
plt.xlabel("AT")
plt.ylabel("PE")
plt.title("Univariate Linear Regression (Normal Equation)")
plt.legend()
plt.show()


### 4.2 Multivariate linear regression (Normal Equation, no scikit-learn)

We now predict electrical power output using four input features:

$$
\hat{y} = \theta_0 + \theta_1 AT + \theta_2 V + \theta_3 AP + \theta_4 RH
$$

Let $X \in \mathbb{R}^{m \times 4}$ contain the columns $[AT, V, AP, RH]$.  
We add a bias (intercept) term to form the design matrix $X_b$:

$$
X_b =
\begin{bmatrix}
1 & AT^{(1)} & V^{(1)} & AP^{(1)} & RH^{(1)}\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
1 & AT^{(m)} & V^{(m)} & AP^{(m)} & RH^{(m)}
\end{bmatrix}
$$

Normal equation:
$$
\theta = (X_b^T X_b)^{-1} X_b^T y
$$

**Important (no data leakage):**  
If you standardize features, compute mean and std **on the training set only**, then apply the same transform to the test set.

**TODO requirements**
- Standardize the feature columns (train statistics only) *(recommended)*
- Add the bias column to build $X_b$
- Compute $\theta$ using the normal equation (with matrix inverse)
- Evaluate using RMSE and $R^2$
- Plot **Predicted vs True** on the test set, and include the reference line $y=x$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score  # metrics are fine; model must be from scratch

# Shapes
m_train, n_features = Xmul_train.shape
m_test = Xmul_test.shape[0]

# Ensure column vectors
y_train_col = y_train.reshape(-1, 1)
y_test_col  = y_test.reshape(-1, 1)

# -----------------------------
# TODO(1): Standardize features (recommended)
# Compute mean/std from TRAIN only, then transform train and test.
#
# Hint:
#   mu = ...
#   sigma = ...
#   Xs_train = ...
#   Xs_test  = ...
# -----------------------------
Xs_train = None
Xs_test  = None

# -----------------------------
# TODO(2): Add bias column to build design matrices
# Xb = [1, standardized_features...]
# -----------------------------
Xb_train = None
Xb_test  = None

# -----------------------------
# TODO(3): Normal equation using matrix inverse
# theta = (Xb^T Xb)^{-1} Xb^T y
#
# Hint:
#   A = Xb_train.T @ Xb_train
#   b = Xb_train.T @ y_train_col
#   theta = np.linalg.inv(A) @ b
# -----------------------------
theta = None

# Predict
y_pred_multi = Xb_test @ theta

# Metrics
print("Multivariate (from scratch) RMSE:", rmse(y_test_col, y_pred_multi))
print("Multivariate (from scratch) R^2:", r2_score(y_test_col, y_pred_multi))
# -----------------------------
# Plot: Predicted vs True (test set)
# -----------------------------
plt.figure(figsize=(6, 5))
plt.scatter(y_test_col, y_pred_multi, alpha=0.7, label="Predictions")

# Reference line y = x
minv = float(min(y_test_col.min(), y_pred_multi.min()))
maxv = float(max(y_test_col.max(), y_pred_multi.max()))
plt.plot([minv, maxv], [minv, maxv], linewidth=2, label="Perfect prediction (y = x)", color="orange")

plt.xlabel("True PE")
plt.ylabel("Predicted PE")
plt.title("Multivariate Linear Regression: Predicted vs True (test set)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


### 4.3 Compare models visually (Predicted vs True)

We compare two regression models trained on the **cleaned dataset** `df_work`:

- **Univariate**: $AT \rightarrow PE$
- **Multivariate**: $[AT, V, AP, RH] \rightarrow PE$

For each model we report:
- RMSE
- $R^2$

And we plot **Predicted vs True** on the test set, including the reference line $y=x$  
(perfect predictions would lie exactly on this line).


In [ ]:
# --- Ensure consistent shapes ---
y_true = np.asarray(y_test_col).reshape(-1, 1)
y_uni  = np.asarray(y_pred_uni).reshape(-1, 1)
y_mul  = np.asarray(y_pred_multi).reshape(-1, 1)

# --- Metrics ---
rmse_uni = rmse(y_true, y_uni)
r2_uni   = r2_score(y_true, y_uni)

rmse_mul = rmse(y_true, y_mul)
r2_mul   = r2_score(y_true, y_mul)

print(f"Univariate (AT)            -> RMSE: {rmse_uni:.3f}, R^2: {r2_uni:.3f}")
print(f"Multivariate (AT,V,AP,RH)  -> RMSE: {rmse_mul:.3f}, R^2: {r2_mul:.3f}")

# --- Plot: Predicted vs True (both models in one figure) ---
plt.figure(figsize=(6, 5))
plt.scatter(y_true, y_uni, alpha=0.6, color="red",   label="Univariate (AT)")
plt.scatter(y_true, y_mul, alpha=0.6, color="green", label="Multivariate (AT, V, AP, RH)")

minv = float(min(y_true.min(), y_uni.min(), y_mul.min()))
maxv = float(max(y_true.max(), y_uni.max(), y_mul.max()))
plt.plot([minv, maxv], [minv, maxv], linewidth=2, color="orange", label="Perfect prediction (y = x)")

plt.xlabel("True PE")
plt.ylabel("Predicted PE")
plt.title("Predicted vs True — Univariate vs Multivariate (test set)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 5) Compare data quality: corrupted raw vs cleaned vs original

This is the key scientific habit in applied ML:

- define a **baseline**
- change **one factor**
- measure the effect on the **same test set**

Here we keep the **model and evaluation protocol fixed** (same features, same preprocessing pipeline, same train/test split),
and only change the **data quality**:

1) **Corrupted raw**: minimal preparation (just make it runnable)
2) **Corrupted cleaned**: your cleaned dataset (`df_work`)
3) **Original**: the uncorrupted reference dataset

We compare performance using **RMSE** and **$R^2$** on the same test set.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

FEATURES = ["AT", "V", "AP", "RH"]
TARGET = "PE"

# --- Helper: numeric coercion (reuse your existing function if already defined) ---
def coerce_numeric_series(s: pd.Series) -> pd.Series:
    s2 = s.astype(str).str.strip()
    s2 = s2.replace({"N/A": np.nan, "na": np.nan, "NA": np.nan, "": np.nan, "nan": np.nan})
    s2 = s2.str.replace(",", ".", regex=False)
    return pd.to_numeric(s2, errors="coerce")

def prepare_numeric(df_any: pd.DataFrame) -> pd.DataFrame:
    d = df_any.copy()
    for c in FEATURES + [TARGET]:
        d[c] = coerce_numeric_series(d[c])
    return d

def rmse_np(y_true, y_pred) -> float:
    y_true = np.asarray(y_true).reshape(-1, 1)
    y_pred = np.asarray(y_pred).reshape(-1, 1)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

# --- Load the three scenarios ---
df_corrupt_raw = pd.read_csv("power_plant_corrupt.csv")
df_corrupt_raw = prepare_numeric(df_corrupt_raw)

# "Raw baseline": only median-impute features so the pipeline can run
df_corrupt_raw_base = df_corrupt_raw.copy()
for c in FEATURES:
    df_corrupt_raw_base[c] = df_corrupt_raw_base[c].fillna(df_corrupt_raw_base[c].median())

df_cleaned = df_work.copy()  # your cleaned dataset
df_original = pd.read_csv("power_plant_clean.csv")  # uncorrupted reference
df_original = prepare_numeric(df_original)

# --- Ensure comparable indexing/length ---
n = min(len(df_corrupt_raw_base), len(df_cleaned), len(df_original))
df_corrupt_raw_base = df_corrupt_raw_base.iloc[:n].reset_index(drop=True)
df_cleaned = df_cleaned.iloc[:n].reset_index(drop=True)
df_original = df_original.iloc[:n].reset_index(drop=True)

# --- Same split indices for fairness ---
idx = np.arange(n)
idx_train, idx_test = train_test_split(idx, test_size=0.2, random_state=RANDOM_STATE)

# Fixed model+preprocessing for all scenarios
pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
    ("lr", LinearRegression())
])

def fit_eval(df_any: pd.DataFrame, name: str):
    X = df_any[FEATURES].to_numpy()
    y = df_any[[TARGET]].to_numpy()

    Xtr, Xte = X[idx_train], X[idx_test]
    ytr, yte = y[idx_train], y[idx_test]

    pipe.fit(Xtr, ytr.ravel())
    pred = pipe.predict(Xte).reshape(-1, 1)

    return {
        "Scenario": name,
        "RMSE": rmse_np(yte, pred),
        "R^2": float(r2_score(yte, pred)),
    }

rows = [
    fit_eval(df_corrupt_raw_base, "Corrupted raw (minimal)"),
    fit_eval(df_cleaned, "Corrupted cleaned (df_work)"),
    fit_eval(df_original, "Original (uncorrupted)"),
]

res = pd.DataFrame(rows)

# Keep a pedagogical order (not sorted)
order = ["Corrupted raw (minimal)", "Corrupted cleaned (df_work)", "Original (uncorrupted)"]
res["Scenario"] = pd.Categorical(res["Scenario"], categories=order, ordered=True)
res = res.sort_values("Scenario").reset_index(drop=True)

display(res)

# --- Visualization: RMSE bars + annotate RMSE and R^2 ---
plt.figure(figsize=(8, 4.5))

colors = ["#d62728", "#2ca02c", "#1f77b4"]  # red, green, blue
bars = plt.bar(res["Scenario"].astype(str), res["RMSE"], color=colors)

plt.ylabel("RMSE (lower is better)")
plt.title("Impact of data quality on model performance (same split, same model)")
plt.xticks(rotation=15, ha="right")

# --- FIX: add headroom on y-axis ---
y_max = res["RMSE"].max()
plt.ylim(0, y_max * 1.20)   # 20% headroom for annotations

# --- Annotate bars (slightly above each bar) ---
for bar, rmse_val, r2_val in zip(bars, res["RMSE"], res["R^2"]):
    h = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        h + 0.05 * y_max,   # offset ABOVE bar, relative to data scale
        f"RMSE={rmse_val:.2f}\nR²={r2_val:.3f}",
        ha="center",
        va="bottom",
        fontsize=9
    )

plt.grid(True, axis="y", alpha=0.25)
plt.tight_layout()
plt.show()


## Final remarks

In this lab you built and evaluated a complete machine learning pipeline, from raw data inspection to model evaluation.  
You saw that improving **data quality** can have a larger impact on performance than changing the model itself.  
You also observed how adding relevant features improves predictions when models are evaluated fairly on the same test set.  
The workflow you practiced here - understand the data, preprocess carefully, train, and evaluate - is central to all applied machine learning.
